## This assignment may be worked individually or in pairs. 
## Enter your name/names here:
    

In [1]:
#names here
#Abhishek Dayal
#Nathan Daniel

# Assignment 2: Naive Bayes and KNN classifier

In this assignment you'll implement the Naive Bayes and KNN classifiers to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the same Diabetic Retinopathy data set which was used in the previous assignment on decision trees. The implementation details are up to you but, generally it is a good idea to divide your code up into helper functions.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
from random import shuffle

Read the data from a CSV file. You may choose to store it any any format you wish, like a Pandas dataframe, or any other data structure you'd like.

In [3]:
def get_data(filename):
    data = []
#     your code goes here
    data = pd.read_csv(filename, header=None)
    return data

## Part 1: Naive Bayes Classifier

Naive Bayes (NB) classifier is a simple probabilistic classifier that is based on applying the Bayes' theorem and assumes a strong (naive) independence between features. The Diabetic Retinopathy data set contains both categorical and continuous features. Dealing with categorical features has been discussed in detail in class. Continuous attributes, on the other hand, are more interesting to handle. Most commonly, this is done by assuming normal probability distribution over the feature values or by binning the attribute values in a fixed number of bins. In this assignment you'll be implementing the binning approach. For each continuous attribute, you'll construct 3 equal sized bins. For example, feature 5 ranges from `[1 - 120]` the 3 bins that you'll construct will be `[1 - 40]`, `[41 - 80]`, `[81 - 120]`.

Q1. Implement a Naive Bayes classifier. Measure the accuracy of your classifier using 5-fold cross validation and display the confusion matrix. Also print the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [4]:
#PREP
# bin continuous attributes (needs to be done with subset of df that is training data)
def bin_df(df):
    bins_dict = dict()
    for c in range(2,18):
        out, bins = pd.cut(df[c], 3, retbins=True, labels=[1,2,3])
        bins_list = list(bins)
        bins_list[0] = np.iinfo(np.int32).min
        bins_list[-1] = np.iinfo(np.int32).max
        bins_dict[c] = bins_list
        
        df[c] = out

    df.head()
    return (df, bins_dict)

In [5]:
def bin_test(df, bins_dict):
    # bins_dict (K,V) = (feature #, bin threshold list)
    binned_df = df.copy()
    for c in range(2,18):
        ii = pd.IntervalIndex.from_breaks(bins_dict[c])
        column_binned = pd.cut(binned_df[c].to_list(), ii)
        column_binned.categories = [1,2,3]
        binned_df[c] = column_binned
    
#    binned_df.head()
    return binned_df

In [6]:
def nb_classify(test_record, training_df):
    #CLASSIFY
    # maybe use groupBy?
    # calculate ~probability of 1
        # P(record[0]| class=1) * ... * P(record[18]| class=1) * P(class=1)
        
    prior_1 = training_df[training_df[19] == 1].count()[0]/training_df.shape[0]
    prior_0 = training_df[training_df[19] == 0].count()[0]/training_df.shape[0]

    groups = training_df.groupby(training_df[19])
    g1 = groups.get_group(1)
    g0 = groups.get_group(0)
    do_laplace = [0 for x in range(18)]

    for i in range(18):
        # Abhishek the line below is what we used yesterday, but since record is an int with the for each loop
            # being used right now, this is an error.
        rval = test_record[i]
        
        counts_given_1 = g1[g1[i] == rval].count()[0]
        counts_given_0 = g0[g0[i] == rval].count()[0]
        denom_1 = g1.shape[0]
        denom_0 = g0.shape[0]
        
        # number of different values that attribute i can take. add this to denom for laplace
        unique = len(training_df[i].unique())
#         print("une value for column{col}: {num}".format(col=i, num=unique))
        
        if counts_given_1 == 0 or counts_given_0 == 0:
            counts_given_1 += 1
            counts_given_0 += 1
            denom_1 += unique
            denom_0 += unique
        prior_1 *= counts_given_1 / denom_1
        prior_0 *= counts_given_0 / denom_0
    # calculate ~probability of 0
        # P(record[0]| class=0) * ... * P(record[18]| class=0) * P(class=0)
#     for i in range(0,18):
#         rval = test_record[i]
#         counts = 
#         if counts == 0:
#             do_laplace[i] = 1
#         prior_0 *= counts / g0.shape[0]
    # return greater of ~probabilities
    
    return 1 if prior_1 > prior_0 else 0

In [7]:
# your code goes here

df = get_data('messidor_features.txt')
df = df.sample(frac=1).reset_index(drop=True)

# TODO: trim off class label and hold separately

sum_acc = 0

#OUTPUT
# 5-fold cross validation

data_len = df.shape[0]
interval = (int)(data_len / 5)

# print(data_len)
# print(interval)

confusion_matrix = np.array([[0,0],[0,0]], np.int32)


for i in range(0, data_len - interval, interval):
    cur_matrix = np.array([[0,0],[0,0]], np.int32)
    # partition data into train_set and test_set
    train_set = df[0:i].append(df[i+interval:])
    # test_set is deep copy
    test_set = df[i:i+interval]
    
    #print(test_set.head())
    
    # get binned df (tuple)
    binned_train_set, bins = bin_df(train_set)
    
    # binned_test_set is a shallow copy
    binned_test_set = bin_test(test_set, bins)
    
    
    # Abhishek maybe take a look at what this prints! It looks good overall but some values 
    # are coming in as 0s so I guess those were the outliers. Do we wanna just adjust
    # all the buckets to have high maxs and low mins so they get put in either 1 or 3?
#     print(binned_test_set.head())
#    tester_record = pd.DataFrame([[1,   1,  2,  2,  2,  2,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1],
#                                 [3,   1,  2,  2,  2,  2,  1,  1,  2,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1]])
#    print(classify(binned_train_set, tester_record.loc(0)))

    # run testing set (!!!! This should be replaced)
    
    predicted_values = binned_test_set.apply(nb_classify, args=(binned_train_set,), axis=1)
#     print("predicted_values", predicted_values, type(predicted_values))
    for idx, pred in predicted_values.items():
#         print("binned_test_set", binned_test_set.iloc(idx))
#         print("Predicted %i for record %i" %(idx, pred))
#         print("record %i: " % idx, binned_test_set.iloc[idx] )
        actl = binned_test_set.loc[idx][19]
#         print("Actual: ", actl)
        cur_matrix[actl][pred] += 1
    print("Confusion Matrix for fold %i\n" % (i//interval + 1), cur_matrix)
    confusion_matrix += cur_matrix
    
#     for record in binned_test_set:
#         print("guess:", classify(binned_train_set, record), "actual:", record[19])
        
    # print and factor in new accuracy (store numbers for misplaced testing figures for confusion matrix)
    # TN FP
    # FN TP
    
    # TODO make "accuracy" = total of 
    accuracy = (cur_matrix[0][0] + cur_matrix[1][1])/len(predicted_values)
    sum_acc += accuracy

    
print("Average Accuracy is: ", sum_acc/5)
# output confusion matrix

print("\nFinal Confusion Matrix\n", confusion_matrix)

predicted_ones = np.sum(confusion_matrix, dtype=np.int32, axis=0)[1]
actually_ones  = np.sum(confusion_matrix, dtype=np.int32, axis=1)[1]
correctly_predicted_ones = confusion_matrix[1][1]
# print precision/recall of class 1
precision = correctly_predicted_ones/predicted_ones
recall = correctly_predicted_ones/actually_ones

print("Class 1 Precision: %f and Recall: %f" % (precision, recall))



Confusion Matrix for fold 1
 [[71 36]
 [65 58]]
Confusion Matrix for fold 2
 [[72 31]
 [59 68]]
Confusion Matrix for fold 3
 [[70 38]
 [51 71]]
Confusion Matrix for fold 4
 [[75 37]
 [61 57]]
Confusion Matrix for fold 5
 [[77 32]
 [55 66]]
Average Accuracy is:  0.5956521739130436

Final Confusion Matrix
 [[365 174]
 [291 320]]
Class 1 Precision: 0.647773 and Recall: 0.523732



## Part 2: K Nearest Neighbor (KNN) Classifier

The KNN classifier consists of two stages:-
- In the training stage, the classifier takes the training data and simply memorizes it
- In the test stage, the classifier compares the test data with the training data and simply returns the maximum occuring label of the k nearest data points.

The distance calculation method is central to the algorithm, typically Euclidean distance is used but other distance metrics like Manhattan distance can also be used. In this assignment you'll be implementing the classifier using the Euclidean distance metric. It is important to note that Euclidean distance is very sensitive to the scaling of different attributes hence, before you can build your classifier you have to normalize the values of each feature in the data set.

Q2. Normalize the dataset so that each feature value lies between `[0-1]`.

In [8]:
# your code goes here
def normalize_df(df):
    normalized_df=(df-df.min())/(df.max()-df.min())
    return normalized_df

Q3. Build your KNN classifier. 

In [9]:
# your code goes here
def knn_classify(record, df, k):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-c19df405dd4f>, line 3)

Q4. Find the best value of k for this data. Try k ranging from 1 to 10. For each k value, use a 5-fold cross validation to evaluate the accuracy with that k. In each fold of CV, divide your data into a training set and a validation set. Print out the best value of k and the accuracy achieved with that value. Return the best value of k. 

In [ ]:
# your code goes here
def find_best_k(df):
    data_len = df.shape[0]
    interval = (int)(data_len / 5)
    
    best_accuracy = 0
    best_k = 1

    for i in range(0, data_len - interval, interval):

        # partition data into train_set and validation_set
        train_set = df[0:i].append(df[i+interval:])
        # validation_set is deep copy
        validation_set = df[i:i+interval]
        
        for k in range(1, 10):
            # call KNN classifier with k, train_set, and validation_set
            # compute accuracy of results
            accuracy = 0
            if (accuracy >= best_accuracy):
                best_accuracy = accuracy
                best_k = k
        
        
    print("Best k:", best_k, "Accuracy Achieved:", best_accuracy)
    return best_k

Q5. Now measure the accuracy of your classifier using 5-fold cross validation. In each fold of this CV, divide your data into a training set and a test set. The training set should get sent through your code for Q4, resulting in a value of k to use. Using that k, calculate an accuracy on the test set. You will average the accuracy over all 5 folds to obtain the final accuracy measurement. Print the accuracy as well as the precision and recall for class label 1 (patients that have been diagnosed with the disease).

In [ ]:
# your code goes here

df = get_data('messidor_features.txt')
sum_acc = 0

# 5-fold cross validation

data_len = df.shape[0]
interval = (int)(data_len / 5)

# print(data_len)
# print(interval)

for i in range(0, data_len - interval, interval):

    # partition data into train_set and test_set
    train_set = df[0:i].append(df[i+interval:])
    # test_set is deep copy
    test_set = df[i:i+interval]
    
    norm_train_set = normalize_df(train_set)
    norm_test_set = normalize_df(test_set)
    
    k = find_best_k(norm_train_set)
    # call KNN classifier with k, norm_train_set, norm_test_set
    # compute accuracy of result
    accuracy = 0
    sum_acc += accuracy

    
print("Average Accuracy is: ", sum_acc/5)